In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier
import joblib

In [2]:
data = pd.read_csv('shopping_behavior.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'shopping_behavior.csv'

In [12]:
df = data.drop(labels=["Customer ID","Review Rating","Shipping Type","Promo Code Used"], axis = 1)
df.head(3)

Age Gender Item Purchased  Category  Purchase Amount (USD)       Location  \
0   55   Male         Blouse  Clothing                     53       Kentucky   
1   19   Male        Sweater  Clothing                     64          Maine   
2   50   Male          Jeans  Clothing                     73  Massachusetts   

  Size   Color  Season Subscription Status Discount Applied  \
0    L    Gray  Winter                 Yes              Yes   
1    L  Maroon  Winter                 Yes              Yes   
2    S  Maroon  Spring                 Yes              Yes   

   Previous Purchases Payment Method Frequency of Purchases  
0                  14          Venmo            Fortnightly  
1                   2           Cash            Fortnightly  
2                  23    Credit Card                 Weekly

In [13]:
names = df.columns
types = df.dtypes

scaler = MinMaxScaler()

label_encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Save the label encoders
for col, le in label_encoders.items():
    joblib.dump(le, f'./encoder/label_encoder_{col.lower()}.pkl')


In [14]:
X_trend = df[['Age', 'Gender', 'Category', 'Location',  'Season']]
Y_trend = df['Item Purchased']
X_trend_scaled = scaler.fit_transform(X_trend)
model_trend = RandomForestClassifier(random_state=42)
model_trend.fit(X_trend_scaled, Y_trend)


RandomForestClassifier(random_state=42)

In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
def predict_trend_item(age, gender, category, location, season):
    new_customer_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Category': [category],
        'Location': [location],
        'Season': [season]
    })
    for col in ['Age', 'Gender', 'Category', 'Location', 'Season']:
        if new_customer_data[col][0] not in label_encoders[col].classes_:
            print(f"Giá trị '{new_customer_data[col][0]}' không có trong label encoder của cột '{col}'")
            return None
        new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])
    new_customer_data = new_customer_data[['Age', 'Gender', 'Category', 'Location', 'Season']]

    new_customer_data_scaled = scaler.transform(new_customer_data)
    predicted_item = model_trend.predict(new_customer_data_scaled)
    predicted_item_name = label_encoders['Item Purchased'].inverse_transform(predicted_item)
    print("Dự đoán sản phẩm cho khách hàng:", predicted_item_name)
    return predicted_item_name


In [16]:
item = predict_trend_item(30, 'Female', 'Accessories', 'Kentucky','Summer')

Dự đoán sản phẩm cho khách hàng: ['Belt']


In [17]:
import joblib

In [18]:
joblib.dump(model_trend, 'model_trend.pkl')

joblib.dump(scaler, './scaler/scaler.pkl')

['./scaler/scaler.pkl']